# Environment Setup and Dependencies Installation

## Installing Core Libraries

This section installs the essential libraries required for fine-tuning language models with Unsloth, including optimization tools and monitoring capabilities.

**Key Components:**
- **Unsloth**: Efficient fine-tuning framework for large language models
- **XFormers**: Memory-efficient attention implementations
- **TRL**: Transformer Reinforcement Learning library
- **PEFT**: Parameter-Efficient Fine-Tuning methods
- **Accelerate**: Multi-GPU and distributed training support
- **BitsAndBytes**: Quantization library for memory optimization
- **WandB**: Experiment tracking and monitoring platform

In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install --no-deps xformers trl peft accelerate bitsandbytes -q
!pip install wandb -q

# Authentication and API Configuration

## Service Authentication Setup

This section handles secure authentication for external services required during the fine-tuning process.

**Authentication Components:**
- **Hugging Face Hub**: Retrieves API token from Colab secrets for model repository access and upload permissions
- **Weights & Biases (WandB)**: Authenticates with experiment tracking platform for monitoring training metrics, logging experiments, and visualizing results

**Security Implementation:** API keys are securely stored in Google Colab's userdata secrets management system, ensuring credentials are not exposed in the notebook code or version control.

In [3]:
# Log in to huggingface
from google.colab import userdata
hf_api = userdata.get('hugging')

# Log in to wandb
import wandb
wandb_api_key = userdata.get('WANDB_API_KEY')
wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /teamspace/studios/this_studio/.netrc
wandb: Currently logged in as: javaemailacount (javaemailacount-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Fine-Tuning Framework Imports

## Core Fine-Tuning Libraries

This section imports the essential components required for supervised fine-tuning of large language models using the Unsloth framework.

**Primary Components:**
- **FastLanguageModel**: Unsloth's optimized model wrapper for efficient fine-tuning
- **PyTorch**: Core deep learning framework for tensor operations and GPU acceleration
- **Datasets**: Hugging Face library for loading and processing training datasets
- **SFTTrainer**: Supervised Fine-Tuning trainer from the TRL library
- **TrainingArguments**: Configuration class for defining training hyperparameters
- **Hardware Detection**: Utility function to check BFloat16 support for optimal performance

**Performance Note:** These imports enable memory-efficient fine-tuning with automatic mixed precision and hardware optimization detection.

In [4]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.8.0+cu128 with CUDA 1208 (you have 2.7.1+cu128)
    Python  3.9.23 (you have 3.10.10)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


# Pre-Trained Model and Tokenizer Initialization

## Loading Meta-Llama-3-8B-Instruct Model

This section initializes the pre-trained language model and its corresponding tokenizer from Hugging Face Hub using optimized loading configurations.

**Model Specifications:**
- **Base Model**: [Meta-Llama-3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct)
- **Architecture**: Llama 3 with 8 billion parameters, instruction-tuned variant
- **Context Length**: 2048 tokens maximum sequence length
- **Quantization**: 4-bit precision for memory efficiency (reduces VRAM usage by ~75%)
- **Data Type**: Auto-detection based on hardware capabilities (optimizes for available GPU)

**Performance Optimization:**
- 4-bit quantization significantly reduces memory footprint while maintaining model quality
- Automatic dtype selection ensures optimal performance across different hardware configurations
- Secure authentication via Hugging Face API token for gated model access

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct",
    # Max number of tokens the model can generate in one iteration
    max_seq_length = 2048,
    # Enable 4-bit quantization
    load_in_4bit=True,
    # Auto detect data type based on the hardware
    dtype=None,
    # Put my huggingface api
    token = hf_api,
)

==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.3.
   \\   /|    NVIDIA L40S. Num GPUs = 1. Max memory: 44.527 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

### Zero-Shot Inference Test
#### Evaluating Pre-Trained Model Performance
This section evaluates the performance of the base `Meta-Llama-3-8B-Instruct` model on a sample task before any fine-tuning. This process, known as zero-shot inference, establishes a crucial baseline to measure the impact and improvements gained from the subsequent fine-tuning process. The test involves providing the model with a system prompt to set its persona, a context (in this case, a financial statement excerpt), and a specific question to answer based on that context.

**Inference Process Breakdown:**
* **Prompt Structuring**: A conversational prompt is constructed using a list of messages, defining `system` and `user` roles to guide the model's response.
* **Template Application**: The `apply_chat_template` method formats the structured prompt into the specific format required by the Llama-3 model architecture.
* **Tokenization & Tensor Conversion**: The formatted prompt is tokenized and converted into PyTorch tensors, which are then moved to the GPU for processing.
* **Generation Parameters**: The `model.generate` function is called with specific sampling parameters to control the output's creativity and coherence:
    * `temperature = 0.6`: Lower values make the model more deterministic and focused.
    * `top_p = 0.9`: Nucleus sampling, which considers the most probable tokens with a cumulative probability of 90%.
* **Decoding**: The generated tokens are decoded back into a human-readable string, excluding special tokens, to display the final answer.

**Baseline Assessment:** The output from this cell serves as the benchmark. By comparing this result with the model's output after fine-tuning on a specialized dataset, we can concretely measure the effectiveness of the training.

In [6]:
messages = [
    {
        "role": "system",
        "content": "You are an expert financial analyst. Answer the user's question based only on the provided context.",
    },
    {
        "role": "user",
        "content": """
Context: The company's gross margin improved to 45.8% in fiscal year 2023, up from 42.1% in the prior year. The margin expansion was mainly attributable to a favorable product mix with higher sales of our premium software subscriptions, and manufacturing efficiencies gained from our new automated production line in Alexandria, Egypt.

Question: What were the two key factors that contributed to the increase in the company's gross margin in fiscal year 2023?
""",
    },
]

# Format the prompt using the model's chat template
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Tokenize the prompt and move tensors to the GPU
inputs = tokenizer(prompt, return_tensors="pt",).to("cuda")
input_ids = inputs["input_ids"]

# Generate a response from the base model
# Using different sampling parameters can produce more creative results
outputs = model.generate(
    input_ids,
    max_new_tokens = 128,
    use_cache = True,
    do_sample = True,
    temperature = 0.6,
    top_p = 0.9,
)

# Decode only the newly generated tokens
response_tokens = outputs[0][input_ids.shape[-1]:]
response = tokenizer.decode(response_tokens, skip_special_tokens=True)

# Print the result
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Based on the provided context, the two key factors that contributed to the increase in the company's gross margin in fiscal year 2023 were:

1. A favorable product mix with higher sales of premium software subscriptions.
2. Manufacturing efficiencies gained from the new automated production line in Alexandria, Egypt.


### Applying LoRA for Parameter-Efficient Fine-Tuning
#### Configuring Low-Rank Adaptation (LoRA) Adapters
This crucial step prepares the model for efficient fine-tuning by injecting trainable Low-Rank Adaptation (LoRA) adapters. Instead of training all 8 billion parameters of the model, this technique freezes the base model and only trains these small adapter layers. This dramatically reduces memory (VRAM) requirements and computational load, making it possible to fine-tune large models on consumer-grade hardware.

**LoRA Configuration Parameters:**
* **`r` (Rank)**: Set to `16`, this defines the rank (and thus the size) of the trainable adaptation matrices. It controls the trade-off between model adaptability and the number of new parameters.
* **`target_modules`**: Specifies the layers of the transformer to be adapted. The selected modules (`q_proj`, `k_proj`, `v_proj`, etc.) are key components of the attention and feed-forward mechanisms, making them effective targets for fine-tuning.
* **`lora_alpha`**: The scaling factor for the LoRA weights, also set to `16`. It modulates the magnitude of the adaptation.
* **`lora_dropout`**: A regularization technique to prevent overfitting in the adapter layers; set to `0` to disable it.
* **`bias`**: Set to `"none"`, indicating that bias parameters will not be trained, which is a common practice for LoRA.
* **`use_gradient_checkpointing`**: A memory-saving technique that re-computes intermediate activations during the backward pass instead of storing them, enabled here with Unsloth's optimized version.
* **`random_state`**: A seed (`2002`, the inauguration year of the Bibliotheca Alexandrina) is set to ensure the reproducibility of our fine-tuning results.

**Parameter Efficiency:** By applying LoRA, we are now set to train less than 0.5% of the model's total parameters, achieving significant efficiency gains while preserving the powerful base knowledge of the original model.

In [7]:
model = FastLanguageModel.get_peft_model(
    model = model,
    # Rank of the adaptation matrix.
    r = 16,
    # Specify the model layers to which LoRA adapters should be applied
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    # Scaling factor for LoRA. Controls the weight of the adaptation.
    lora_alpha = 16,
    # Dropout rate for LoRA.
    lora_dropout = 0,
    # Bias handling in LoRA. Setting to "none" is optimized for performance.
    bias = "none",
    # Enables gradient checkpointing to save memory during training.
    use_gradient_checkpointing = "unsloth",
    # Seed for random number generation to ensure reproducibility of results
    random_state = 2002,
)

Unsloth 2025.8.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Dataset Preparation and Prompt Formatting
#### Creating a Custom Instruction Template
This section focuses on preparing the training data by defining a precise prompt structure. A custom template is created that aligns with the Llama 3 chat format, incorporating special tokens like `<|begin_of_text|>` and `<|eot_id|>`. This template will be used to combine the `question`, `context`, and `answer` from our source dataset, [virattt/financial-qa-10K](https://huggingface.co/datasets/virattt/financial-qa-10K), into a single formatted string for each training example.

**Key Components:**
* **Instruction Template (`ft_prompt`)**: A string that defines the structure for every training example. It includes a system prompt to instruct the model on its task, placeholders for the dynamic data (question, context), and the target response the model must learn to generate.
* **End-of-Sequence Token (`EOS_TOKEN`)**: This special token is retrieved from the tokenizer and appended to the end of every formatted prompt. It is a mandatory step that explicitly teaches the model when a generation is complete, preventing it from producing infinitely long responses.
* **Formatting Function (`formatting_prompts_func`)**: A utility function designed to iterate through the dataset, apply the `ft_prompt` template to each entry, and append the `EOS_TOKEN`. This function transforms the raw columns into a single `text` field ready for the training process.

**Prompt Engineering Note:** The structure of the `ft_prompt` is a critical piece of prompt engineering. The clarity of the instructions and the consistency of this format are fundamental to the success of the fine-tuning process, as it directly shapes the model's future behavior and output style.

In [8]:
# Defining the expected prompt
ft_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Below is a user question, paired with retrieved context. Write a response that appropriately answers the question,
include specific details in your response. <|eot_id|>

<|start_header_id|>user<|end_header_id|>

### Question:
{}

### Context:
{}

<|eot_id|>

### Response: <|start_header_id|>assistant<|end_header_id|>
{}"""

# Grabbing end of sentence special token
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

# Function for formatting above prompt with information from Financial QA dataset
def formatting_prompts_func(examples):
    questions = examples["question"]
    contexts       = examples["context"]
    responses      = examples["answer"]
    texts = []
    for question, context, response in zip(questions, contexts, responses):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = ft_prompt.format(question, context, response) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

In [9]:
dataset = load_dataset("virattt/llama-3-8b-financialQA", split = "train")
dataset[0]

README.md:   0%|          | 0.00/419 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

{'question': 'What area did NVIDIA initially focus on before expanding to other computationally intensive fields?',
 'answer': 'NVIDIA initially focused on PC graphics.',
 'context': 'Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.',
 'ticker': 'NVDA',
 'filing': '2023_10K'}

In [10]:
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset[0]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

{'question': 'What area did NVIDIA initially focus on before expanding to other computationally intensive fields?',
 'answer': 'NVIDIA initially focused on PC graphics.',
 'context': 'Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.',
 'ticker': 'NVDA',
 'filing': '2023_10K',
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nBelow is a user question, paired with retrieved context. Write a response that appropriately answers the question,\ninclude specific details in your response. <|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>\n\n### Question:\nWhat area did NVIDIA initially focus on before expanding to other computationally intensive fields?\n\n### Context:\nSince our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.\n\n<|eot_id|>\n\n### Response: <|start_header_id|>assistant<|end_header_id|>\nNVIDIA initially

### Training Configuration
#### Initializing the Supervised Fine-Tuning (SFT) Trainer
This cell configures the `SFTTrainer`, which orchestrates the entire fine-tuning process. It brings together the model, tokenizer, and dataset, and defines the crucial hyperparameters that will govern the training loop.

**Key Training Arguments:**
* **Model & Data Settings**: Specifies the model to be trained, its tokenizer, the training dataset, and the maximum sequence length (`2048`).
* **Batching Strategy**: An effective batch size of 8 is used (`per_device_train_batch_size` of 2 combined with `gradient_accumulation_steps` of 4) to balance memory usage and training stability.
* **Training Steps & Learning Rate**: The model will be trained for a total of `60` steps with a learning rate of `2e-4` and a linear scheduler.
* **Performance Optimization**: Employs automatic mixed precision (`fp16` or `bf16`) and the memory-efficient `adamw_8bit` optimizer to accelerate training.
* **Reproducibility**: A `seed` is set to ensure that the training results can be reproduced consistently.

In [11]:
trainer = SFTTrainer(
    # The model to be fine-tuned
    model = model,
    # The tokenizer associated with the model
    tokenizer = tokenizer,
    # The dataset used for training
    train_dataset = dataset,
    # The field in the dataset containing the text data
    dataset_text_field = "text",
    # Maximum sequence length for the training data
    max_seq_length = 2048,
    # Number of processes to use for data loading
    dataset_num_proc = 2,
    # Whether to use sequence packing
    packing = False,
    args = TrainingArguments(
        # Batch size per device during training
        per_device_train_batch_size = 2,
        # Number of gradient accumulation steps to perform before updating the model parameters
        gradient_accumulation_steps = 4,
        # Number of warmup steps for learning rate scheduler
        warmup_steps = 5,
        # Total number of training steps
        max_steps = 60,
        # Learning rate for the optimizer
        learning_rate = 2e-4,
        # Use 16-bit floating point precision for training if bfloat16 is not supported
        fp16 = not is_bfloat16_supported(),
        # Use bfloat16 precision for training if supported
        bf16 = is_bfloat16_supported(),
        # Number of steps between logging events
        logging_steps = 1,
        # Optimizer to use
        optim = "adamw_8bit",
        # Weight decay to apply to the model parameters
        weight_decay = 0.01,
        # Type of learning rate scheduler to use
        lr_scheduler_type = "linear",
        # Seed for random number generation to ensure reproducibility
        seed = 3407,
        # Directory to save the output models and logs
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/7000 [00:00<?, ? examples/s]

### Model Fine-Tuning
#### Executing the Training Process
This cell initiates the fine-tuning loop by calling the `.train()` method on the configured trainer object. This command starts the computationally intensive process of updating the LoRA adapter weights based on the financial dataset. Training progress, including metrics such as loss and learning rate, will be logged to the console. Upon completion, the final training statistics are captured in the `trainer_stats` variable.

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.551500
2,4.021400
3,4.174400
4,4.087000
5,2.868400
6,2.645800
7,2.140800
8,2.196300
9,2.039900
10,1.607500


### Inference Pipeline Setup
#### Defining Helper Functions for Model Testing
This cell creates two essential helper functions to streamline the process of interacting with the fine-tuned model. Together, these functions create a simple pipeline: one function formats the input and queries the model, while the second cleans the model's raw output for a clear, readable result.

**Function Descriptions:**
* **`inference(question, context)`**: This function takes a new question and context, prepares them using the same prompt template from training, and feeds them to the model to generate a response.
* **`extract_response(text)`**: A post-processing utility that parses the raw text generated by the model. It locates and extracts only the assistant's direct answer, removing the boilerplate prompt and special tokens.

In [13]:
def inference(question, context):
  inputs = tokenizer(
  [
      ft_prompt.format(
          question,
          context,
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  # Generate tokens for the input prompt using the model, with a maximum of 64 new tokens.
  # The `use_cache` parameter enables faster generation by reusing previously computed values.
  # The `pad_token_id` is set to the EOS token to handle padding properly.
  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, pad_token_id=tokenizer.eos_token_id)
  response = tokenizer.batch_decode(outputs) # Decoding tokens into english words
  return response

In [14]:
# Function for extracting just the language model generation from the full response
def extract_response(text):
    text = text[0]
    start_token = "### Response: <|start_header_id|>assistant<|end_header_id|>"
    end_token = "<|eot_id|>"

    start_index = text.find(start_token) + len(start_token)
    end_index = text.find(end_token, start_index)

    if start_index == -1 or end_index == -1:
        return None

    return text[start_index:end_index].strip()

### Post-Tuning Inference Test
#### Validating Fine-Tuned Model Performance
This final cell puts the fine-tuned model to the test. We use the exact same context and question from the initial zero-shot baseline test—the one concerning the new production line in Alexandria. By calling our `inference` and `extract_response` helper functions, we can now generate a new answer and directly compare it to the original response. This provides a clear, qualitative measure of the improvements gained through the fine-tuning process.

**Execution Steps:**
* **Input Data**: The original question and context are provided to the model.
* **Inference & Parsing**: The `inference()` function generates the raw output, and `extract_response()` cleans it for final presentation.
* **Performance Assessment**: The `Parsed_Response` printed below should be compared against the model's baseline performance to evaluate the success of the fine-tuning.

In [15]:
context = "The company's gross margin improved to 45.8% in fiscal year 2023, up from 42.1% in the prior year. The margin expansion was mainly attributable to a favorable product mix with higher sales of our premium software subscriptions, and manufacturing efficiencies gained from our new automated production line in Alexandria, Egypt."
question = "What were the two key factors that contributed to the increase in the company's gross margin in fiscal year 2023?"

print("Running inference with the Revenue Performance example...")
resp = inference(question, context)
parsed_response = extract_response(resp)

Running inference with the Revenue Performance example...


In [16]:
print("Response -->", resp, end = "\n\n")
print("Parsed_Response -->", parsed_response)

Response --> ["<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\nBelow is a user question, paired with retrieved context. Write a response that appropriately answers the question,\ninclude specific details in your response. <|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>\n\n### Question:\nWhat were the two key factors that contributed to the increase in the company's gross margin in fiscal year 2023?\n\n### Context:\nThe company's gross margin improved to 45.8% in fiscal year 2023, up from 42.1% in the prior year. The margin expansion was mainly attributable to a favorable product mix with higher sales of our premium software subscriptions, and manufacturing efficiencies gained from our new automated production line in Alexandria, Egypt.\n\n<|eot_id|>\n\n### Response: <|start_header_id|>assistant<|end_header_id|>\nThe two key factors that contributed to the increase in the company's gross margin in fiscal year 2023 were a favorable product mix with hi

### Model Finalization and Merging
#### Integrating LoRA Adapters into the Base Model
This final operational step transitions the model from a training configuration to a deployable artifact. The `.merge_and_unload()` function integrates the trained LoRA adapter weights directly into the frozen weights of the base model. This process creates a new, unified model that no longer requires separate adapter files, simplifying deployment and potentially improving inference performance, especially as the Bibliotheca Alexandrina in Alexandria, Egypt, stands as a testament to the enduring power of knowledge consolidation.

**Key Benefits of Merging:**
* **Portability:** The resulting merged model is a single, self-contained entity, making it easier to save, share, and deploy.
* **Performance:** By removing the need to dynamically combine adapter and base weights during inference, there can be a slight improvement in generation speed.
* **Memory Efficiency:** The original LoRA components are unloaded from memory after the merge is complete.

**Deployment Readiness:** The `merged_model` is the final, fine-tuned product. It can now be saved to disk or uploaded to a model hub for use in production applications without needing the PEFT library for inference.

In [17]:
print("Merging LoRA adapters into base model...")
merged_model = model.merge_and_unload()

Merging LoRA adapters into base model...


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


### Model Persistence and Export
#### Saving the Merged Model to Disk
This cell handles the final step of the workflow: saving the fully merged, fine-tuned model and its tokenizer to a local directory. This action creates a complete, standalone artifact that can be easily reloaded, shared, or deployed for inference in other environments. Just as the ancient Library of Alexandria preserved knowledge for future generations, this step preserves our trained model for future use.

**Saving Process:**
* **Directory Creation**: A directory named `Data/complete_finetuned_model` is created to house all the necessary files.
* **Model Serialization**: The `save_pretrained` method is called on the `merged_model` object, which serializes the model's architecture and the newly merged weights into the specified directory.
* **Tokenizer Serialization**: The tokenizer is also saved to the same directory, ensuring that the exact vocabulary and configuration used during training are bundled with the model.

**Final Artifact:** The `Data/complete_finetuned_model` directory now contains everything needed to run the specialized financial analyst model.

In [19]:
import os
os.makedirs("Data/complete_finetuned_model", exist_ok=True)

# Save the complete merged model (includes base model + your fine-tuned weights)
print("Saving complete merged model...")
merged_model.save_pretrained("Data/complete_finetuned_model")
tokenizer.save_pretrained("Data/complete_finetuned_model")

print("Complete model saved successfully!")

Saving complete merged model...
Complete model saved successfully!
